In [77]:
import praw
import numpy
%matplotlib inline
from pylab import *
from pprint import pprint
import pandas as pd
import simplejson as json
from psaw import PushshiftAPI
import pyspark
import datetime as dt

%config InlineBackend.figure_format='retina'

In [78]:
CLIENT_ID = '5FpjYNUrv93rDqflbNE-6w'
CLIENT_SECRET = '5ZITWnmNd6_qCbPXDB90xmOaIMo9_w'
USER_NAME = 'theonejay96'
SCRIPT_NAME = 'Data\ Science\ Thesis'
LOGIN_PASSWORD = 'vejkyz-hyBfez-4cinte'

reddit =  praw.Reddit(
    client_id= CLIENT_ID ,
    client_secret= CLIENT_SECRET ,
    user_agent= SCRIPT_NAME,
    username= USER_NAME,
    password= LOGIN_PASSWORD)

api = PushshiftAPI()

In [205]:
df = pd.read_pickle('/Users/jakobschlierf/Desktop/Master/Thesis/Files/2021-01-02_cross_to_temp.pickle')

In [243]:
df.head()

,id,url,title,subreddit,subreddit_id,subreddit_subscribers,num_crossposts,created_utc,author,created
15,l9fzaw,https://www.reddit.com/r/CovidVaccinated/comme...,Struggling but hoping it will be worth it! 29y...,CovidVaccinated,t5_3gppqs,6109,1,1612111229,Chi2Wisco,1.612104e+09
193,l742hl,https://www.reddit.com/r/CovidVaccinated/comme...,2nd Dose Side Effects 27 y/o F,CovidVaccinated,t5_3gppqs,5042,1,1611857012,aaa93,1.611850e+09
27,l9kyht,https://www.newsweek.com/fourth-covid-case-iow...,Fourth COVID case reported at Iowa Capitol whe...,Coronavirus,t5_2x4yx,2365433,1,1612124327,shallah,1.612117e+09
71,l9gwu3,https://www.cbsnews.com/news/fema-pentagon-rea...,"FEMA asks Pentagon to ready as many as 10,000 ...",Coronavirus,t5_2x4yx,2365401,1,1612113737,Baron-Munc,1.612107e+09
93,l9fmlm,https://www.bbc.co.uk/news/live/uk-55876866,UK reaches record number of daily vaccinations...,Coronavirus,t5_2x4yx,2365399,5,1612110252,callum2703,1.612103e+09


In [242]:
t.head()

,url,num_crossposts,id
0,http://news.sky.com/story/covid-19-rejected-co...,1,l99n5d
1,http://www.politico.com/news/2021/01/30/biden-...,2,l9ibz5
2,https://abcnews.go.com/amp/US/women-20s-dresse...,1,loj74l
3,https://archive.vn/ODFJF,4,ls4blg
4,https://babylonbee.com/news/fauci-spins-wheel-...,1,l9evme


In [241]:
# Code block to find crosspost children from all links
df = df[df['num_crossposts'] > 0]
t = df.groupby(['url']).agg({'num_crossposts': 'sum', 'id':  'max'}).reset_index()
urls = list(t['url'])

In [255]:
df = pd.DataFrame(columns=['id', 'url', 'title','subreddit', 'subreddit_id', 'subreddit_subscribers',
        'num_crossposts', 'crosspost_parent', 'created_utc', 'author'])
for i in urls:
    gen2 = api.search_submissions(
    url = i ,
    filter=[ 'id', 'url', 'title', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
        'num_crossposts', 'crosspost_parent', 'created_utc', 'author'],
    limit = 100) #limit to avoid going over the rate limit.
    results2 = list(gen2)
    temp = pd.DataFrame([thing.d_ for thing in results2])
    df = pd.concat([df, temp])

df2 = df[df['num_crossposts'] > 0]
df3 = df[df['num_crossposts'] == 0]

#merge based on crosspost_parent. for remainder, merge based on link & num_crossposts

/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


KeyboardInterrupt: 

In [254]:
df.to_pickle('../../Files/test_child.pickle')

In [251]:
df3.head(10)

,id,subreddit,subreddit_id,subreddit_subscribers,num_crossposts,crosspost_parent,created_utc,author,created
0,l99xs8,CoronavirusUK,t5_2f3oes,83326,0,t3_l99n5d,1612089642,dayus9,1.612082e+09
2,l99mtt,ukpolitics,t5_2qhcv,357954,0,NaN,1612088223,ManiacalFanSegments,1.612081e+09
0,l9iilg,conservatives,t5_2qj2e,71522,0,t3_l9ibz5,1612117908,Pee_Nut_Pup,1.612111e+09
1,l9ic6c,NEWPOLITIC,t5_2pwz1a,3361,0,t3_l9ibz5,1612117449,Pee_Nut_Pup,1.612110e+09
3,l9i4ba,politics,t5_2cneq,7309728,0,NaN,1612116893,mancan123able,1.612110e+09
0,lojj4v,antivaccine,t5_2wm9w,11622,0,t3_loj74l,1613863509,diirtnap,1.613856e+09
2,lo83te,nottheonion,t5_2qnts,18840374,0,NaN,1613830993,NiceRemove7,1.613824e+09
3,lo815e,nottheonion,t5_2qnts,18840355,0,NaN,1613830751,NiceRemove7,1.613824e+09
4,lo804k,nottheonion,t5_2qnts,18840350,0,NaN,1613830655,NiceRemove7,1.613823e+09
5,lnz0yr,neutralnews,t5_2ql83,93401,0,NaN,1613794830,BrightProfessional8,1.613788e+09


In [250]:
df2.head()

,id,subreddit,subreddit_id,subreddit_subscribers,num_crossposts,crosspost_parent,created_utc,author,created
1,l99n5d,Coronavirus,t5_2x4yx,2365369,1,NaN,1612088268,ManiacalFanSegments,1.612081e+09
2,l9ibz5,NoNewNormal,t5_2q5yp2,34540,2,NaN,1612117435,Pee_Nut_Pup,1.612110e+09
1,loj74l,DebateVaccines,t5_3jwf7,2525,1,NaN,1613862530,diirtnap,1.613855e+09
4,ls4blg,LockdownSkepticism,t5_2iokxd,33966,4,NaN,1614251713,dankseamonster,1.614245e+09
10,l9evme,NoNewNormal,t5_2q5yp2,34518,1,NaN,1612108140,tarkay,1.612101e+09


In [ ]:
df3 = df3['crosspost_parent'].notna().apply(lambda x: x)
df3.join(df2, on)

In [181]:
results = list(gen)
print(f'subreddit: {i}, number of results: {len(results)}')
temp = pd.DataFrame([thing.d_ for thing in results])
t = temp[(temp['num_crossposts'] > 0)]

subreddit: Vaccines, number of results: 190


In [167]:
t = temp.groupby(['url']).count()

In [142]:
gen2 = api.search_submissions(
    url = temp['url'] ,
    filter=[ 'id','subreddit', 'subreddit_id', 'subreddit_subscribers',
        'num_crossposts', 'crosspost_parent', 'created_utc', 'author'],
    limit = limit) #limit to avoid going over the rate limit.
results2 = list(gen2)
temp2 = pd.DataFrame([thing.d_ for thing in results2])

In [195]:
gen3 = api.search_submissions(
    url = 'https://www.reddit.com/r/VACCINES/comments/l93l50/about_vaccine/',
    filter=['id', 'url', 'title',  'subreddit', 'subreddit_id', 'subreddit_subscribers',
        'num_crossposts', 'created_utc', 'author'],
    limit = limit) #limit to avoid going over the rate limit.

results3 = list(gen3)

temp3 = pd.DataFrame([thing.d_ for thing in results3])

/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


In [196]:
temp3

,author,created_utc,id,num_crossposts,subreddit,subreddit_id,subreddit_subscribers,title,url,created
0,Monkeydo111,1612064319,l93l50,0,VACCINES,t5_2r9k3,4162,About vaccine,https://www.reddit.com/r/VACCINES/comments/l93...,1.612057e+09


In [2]:
def importance(df): #calculate the importance (similar to tf-idf)
    # OPEN
    # - make work with Crosspost from
    imp = df.groupby(['crosspost_from']).agg({'subreddit': 'count', 'count': 'sum', 'crosspost_from_num' : 'sum'})
    imp = imp.rename(columns ={'count': 'total'}) 
    imp.drop(['subreddit'], axis=1, inplace=True)
    df2 = df.join(imp, on='crosspost_from', how='left')
    return df2

In [75]:
df2 = pd.read_pickle('../../Files/2021-01-02.pickle')

In [68]:
df3 = df2[df2['crosspost_parent_list'].str.len() == 0]

In [76]:
df2.head()

,index,subreddit,subreddit_id,subreddit_subscribers,crosspost_parent,crosspost_parent_list,created_utc,author,created,t,crosspost_from,crosspost_from_id,crosspost_from_subs,crosspost_from_num
0,0,DebateVaccines,t5_3jwf7,2347,t3_l9iiqp,"[{'all_awardings': [], 'allow_live_comments': ...",1612117931,[deleted],1.612111e+09,"{'all_awardings': [], 'allow_live_comments': F...",conspiracy,t5_2qh4r,1455036,1
1,3,DebateVaccines,t5_3jwf7,2325,t3_l81oft,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,1612056522,doingbasiclifeprep,1.612049e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,IAmA,t5_2qzb6,20780922,1
2,15,DebateVaccines,t5_3jwf7,2317,t3_l7apag,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,1611915334,EuCleo,1.611908e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,Coronavirus,t5_2x4yx,2363477,1
3,17,DebateVaccines,t5_3jwf7,2314,t3_l77ln3,"[{'all_awardings': [], 'allow_live_comments': ...",1611865305,dannylenwinn,1.611858e+09,"{'all_awardings': [], 'allow_live_comments': F...",ForUnitedStates,t5_2i3wxn,2757,26
4,18,DebateVaccines,t5_3jwf7,2314,t3_l746cr,"[{'all_awardings': [], 'allow_live_comments': ...",1611857406,dannylenwinn,1.611850e+09,"{'all_awardings': [], 'allow_live_comments': F...",ForUnitedStates,t5_2i3wxn,2754,17


In [ ]:
submission = reddit.submission('t3_p8ibgd')
submission.title

In [40]:
r['crosspost_parent_list'][0]

IndexError: list index out of range

In [26]:
df2 = df2[df2['crosspost_parent'].notna()].reset_index()

In [52]:

df2.to_pickle(f'../../Files/test.pickle')

IndexError: list index out of range

# PySpark Test

In [6]:
from pyspark import SparkConf, SparkContext 
sc = SparkContext("local[4]", 'spark_test')





ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=spark_test, master=local[4]) created by __init__ at /tmp/ipykernel_320226/2751826978.py:2 

In [36]:
lines = sc.textFile("Files/RS_2005-07.bz2", )

In [42]:
# lines.map(lambda x: x[0]).take(1)
lines.take(1)

['{"archived":true,"author":"zaz","author_flair_background_color":null,"author_flair_css_class":null,"author_flair_richtext":[],"author_flair_text":null,"author_flair_text_color":null,"author_flair_type":"text","brand_safe":true,"can_gild":true,"contest_mode":false,"created_utc":1120181161,"distinguished":null,"domain":"cnn.com","edited":false,"gilded":0,"hidden":false,"hide_score":false,"id":"204","is_crosspostable":true,"is_reddit_media_domain":false,"is_self":false,"is_video":false,"link_flair_css_class":null,"link_flair_richtext":[],"link_flair_text":null,"link_flair_text_color":"dark","link_flair_type":"text","locked":false,"media":null,"media_embed":{},"no_follow":false,"num_comments":0,"num_crossposts":0,"over_18":false,"parent_whitelist_status":"all_ads","permalink":"\\/r\\/reddit.com\\/comments\\/204\\/cnncom_dj_admits_false_tale_about_missing_teen\\/","rte_mode":"markdown","score":4,"secure_media":null,"secure_media_embed":{},"selftext":"","send_replies":true,"spoiler":false,